In [1]:
import pandas as pd
import json
import sqlite3
import re


orders_df = pd.read_csv(r"C:\Users\tanni_mbxeveq\Downloads\orders.csv")
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'], dayfirst=True)


with open(r"C:\Users\tanni_mbxeveq\Downloads\users.json", 'r') as f:
    users_data = json.load(f)
users_df = pd.DataFrame(users_data)


with open(r"C:\Users\tanni_mbxeveq\Downloads\restaurants.sql", 'r') as f:
    sql_content = f.read()

pattern = re.compile(r"INSERT INTO restaurants VALUES \((.*?)\);")
matches = pattern.findall(sql_content)
res_list = []
for match in matches:
    parts = [p.strip().strip("'") for p in match.split(',')]
    res_list.append({
        'restaurant_id': int(parts[0]),
        'res_name': parts[1],
        'cuisine': parts[2],
        'rating': float(parts[3])
    })
restaurants_df = pd.DataFrame(res_list)

In [2]:
final_df = pd.merge(orders_df, users_df, on='user_id', how='left')
final_df = pd.merge(final_df, restaurants_df, on='restaurant_id', how='left')

In [3]:
print(f"Gold membership orders: {len(final_df[final_df['membership'] == 'Gold'])}")

print(f"Hyderabad total revenue: {round(final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum())}")

print(f"Distinct users: {final_df['user_id'].nunique()}")

print(f"AOV for Gold members: {round(final_df[final_df['membership'] == 'Gold']['total_amount'].mean(), 2)}")

print(f"High-rated restaurant orders (>=4.5): {len(final_df[final_df['rating'] >= 4.5])}")

top_gold_city = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
print(f"Orders in top Gold city ({top_gold_city}): {len(final_df[(final_df['membership'] == 'Gold') & (final_df['city'] == top_gold_city)])}")

print(f"City with most orders: {final_df['city'].value_counts().idxmax()}")

print(f"Most popular cuisine: {final_df['cuisine'].value_counts().idxmax()}")

top_user = final_df.groupby('name')['total_amount'].sum().idxmax()
print(f"Top spending user: {top_user}")

print(f"Average restaurant rating: {round(final_df['rating'].mean(), 2)}")

Gold membership orders: 4987
Hyderabad total revenue: 1889367
Distinct users: 2883
AOV for Gold members: 797.15
High-rated restaurant orders (>=4.5): 3374
Orders in top Gold city (Chennai): 1337
City with most orders: Bangalore
Most popular cuisine: Mexican
Top spending user: User_1515
Average restaurant rating: 4.05


In [4]:
print(f"Orders in December: {len(final_df[final_df['order_date'].dt.month == 12])}")

print(f"Top revenue restaurant ID: {final_df.groupby('restaurant_id')['total_amount'].sum().idxmax()}")

print(f"Membership distribution:\n{final_df['membership'].value_counts()}")

print(f"Orders > 1000: {len(final_df[final_df['total_amount'] > 1000])}")

print(f"Highest AOV cuisine: {final_df.groupby('cuisine')['total_amount'].mean().idxmax()}")

print(f"Top 3 Restaurants:\n{final_df['res_name'].value_counts().head(3)}")

print(f"Revenue by membership:\n{final_df.groupby('membership')['total_amount'].sum()}")

print(f"Busiest date: {final_df['order_date'].value_counts().idxmax().date()}")

print(f"City with best-rated restaurants: {final_df.groupby('city')['rating'].mean().idxmax()}")

print(f"Order count per cuisine:\n{final_df['cuisine'].value_counts()}")

print(f"Most frequent user ID: {final_df['user_id'].value_counts().idxmax()}")

print(f"Avg rating for Chinese: {round(final_df[final_df['cuisine'] == 'Chinese']['rating'].mean(), 2)}")

print(f"User count per city:\n{users_df['city'].value_counts()}")

print(f"Top revenue month: {final_df.groupby(final_df['order_date'].dt.month)['total_amount'].sum().idxmax()}")

gold_users_pct = (users_df[users_df['membership'] == 'Gold'].shape[0] / users_df.shape[0]) * 100
print(f"Percentage of Gold users: {round(gold_users_pct, 2)}%")

Orders in December: 849
Top revenue restaurant ID: 287
Membership distribution:
membership
Regular    5013
Gold       4987
Name: count, dtype: int64
Orders > 1000: 3589
Highest AOV cuisine: Mexican
Top 3 Restaurants:
res_name
Restaurant_383    36
Restaurant_287    35
Restaurant_231    32
Name: count, dtype: int64
Revenue by membership:
membership
Gold       3975364.89
Regular    4036259.23
Name: total_amount, dtype: float64
Busiest date: 2023-07-08
City with best-rated restaurants: Chennai
Order count per cuisine:
cuisine
Mexican    2581
Italian    2532
Indian     2469
Chinese    2418
Name: count, dtype: int64
Most frequent user ID: 2973
Avg rating for Chinese: 4.1
User count per city:
city
Bangalore    799
Chennai      758
Pune         734
Hyderabad    709
Name: count, dtype: int64
Top revenue month: 3
Percentage of Gold users: 49.3%


In [5]:
final_df

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,res_name,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,2023-05-21,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7
9996,9997,2867,267,2023-08-06,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2
9997,9998,522,420,2023-11-11,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0
9998,9999,319,492,2023-09-08,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0
